In [1]:
import os
from pyairtable import Api
from pyairtable import api
from dotenv import load_dotenv
from fibsem_tools import read
from pyairtable.formulas import match
from cellmap_utils import upsert_image

load_dotenv()

at_api = Api(os.environ['AIRTABLE_API_KEY'])
# image table (switch between prod and test bases in .env)
image_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['IMAGE_TABLE_ID'])
collection_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['COLLECTION_TABLE_ID'])
fibsem_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['FIBSEM_TABLE_ID'])
annotation_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['ANNOTATION_TABLE_ID'])
institution_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['INSTITUTION_TABLE_ID'])

In [ ]:
ds_name = ''
image_name = ''
image_path = ''
image_title = ''

upsert_image(at_api,
                ds_name,
                image_name,
                image_path,
                'em')

In [2]:
zarr_paths = ['aic_desmosome-3.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_ccl81-covid-1.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_hela-bfa.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_hela-22.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_hela-21.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_cos7-11.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_sum159-1.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_mus-nacc-4.zarr/recon-2/labels/inference/segmentations/nuc',
 'jrc_mus-nacc-3.zarr/recon-2/labels/inference/segmentations/nuc',
 'jrc_mus-nacc-2.zarr/recon-2/labels/inference/segmentations/nuc',
 'jrc_ctl-id8-5.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_ctl-id8-4.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_ctl-id8-3.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_hela-nz-1.zarr/recon-2/labels/inference/segmentations/nuc',
 'jrc_mus-granule-neurons-1.zarr/recon-2/labels/inference/segmentations/nuc',
 'jrc_mus-pancreas-3.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_mus-pancreas-2.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_mus-sc-zp105a.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_mus-granule-neurons-2.zarr/recon-2/labels/inference/segmentations/nuc',
 'jrc_mus-granule-neurons-3.zarr/recon-2/labels/inference/segmentations/nuc',
 'jrc_mus-sc-zp104a.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_cos7-1a.zarr/recon-2/labels/inference/segmentations/nuc',
'jrc_cos7-1b.zarr/recon-2/labels/inference/segmentations/nuc',
'jrc_mus-kidney-2.zarr/recon-1/labels/inference/segmentations/nuc',
'jrc_mus-guard-hair-follicle.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_mus-meissner-corpuscle-1.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_mus-kidney.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_fly-acc-calyx-1.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_mus-meissner-corpuscle-2.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_fly-mb-z0419-20.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_ctl-id8-1.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_hela-h89-2.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_hela-h89-1.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_mus-pancreas-1.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_fly-fsb-1.zarr/recon-1/labels/inference/segmentations/nuc',
 'jrc_mus-liver-2.zarr/recon-1/labels/inference/segmentations/nuc']




In [3]:
for p in zarr_paths:
    ds_name = p.split('/')[0].split('.')[0]
    image_name = 'nuc'
    image_path = os.path.join('/nrs/cellmap/data/',ds_name, p)
    image_title = 'Nucleus'

    upsert_image(at_api,
                    ds_name,
                    image_name,
                    image_path,
                    image_title,
                    "ml_segmentation")

HTTPError: ('422 Client Error: Unprocessable Entity for url: https://api.airtable.com/v0/appbcC4evkbKjk9yk/tblR65Z21TGGT4sDw', '{\'type\': \'INVALID_RECORD_ID\', \'message\': \'Value "[object Object]" is not a valid record ID.\'}')

In [4]:
!python3 -m pip install cellmap_utils==0.0.2

  Attempting uninstall: cellmap_utils
    Found existing installation: cellmap-utils 0.0.19
    Uninstalling cellmap-utils-0.0.19:
      Successfully uninstalled cellmap-utils-0.0.19


In [ ]:
from typing import Tuple
from pyairtable import api
from pyairtable.formulas import match
from fibsem_tools import read
import os
import zarr

# upsert image record
from typing import Literal


def upsert_image(
    at_api: api,
    ds_name: str,
    image_name: str,
    image_path: str,
    image_title: str,
    image_type: Literal["human_segmentation", "em"],
    institution : str = "HHMI / Janelia Research Campus"
):
    """Upsert a record to airtable image table.

    Args:
        image_table (api.table.Table): image airtable object to create references.
        ds_name (str): name of the dataset.
        image_name (str): name of the image to upsert.
        image_path (str): image location.
        image_title (str): image title on openorganelle.com.
        image_type (Literal[&#39;human_segmentation&#39;, &#39;em&#39;]): image type
        collection_table (api.table.Table): collation airtable object to create references.
        fibsem_table (api.table.Table): fibsem_imaging airtable object to create references.
        annotation_table (api.table.Table): annotation airtable object to create references.

    Raises:
        ValueError: raise value error if multiple records with the same location and name are found in the image table.
    """
    
    image_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['IMAGE_TABLE_ID'])
    collection_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['COLLECTION_TABLE_ID'])
    fibsem_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['FIBSEM_TABLE_ID'])
    annotation_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['ANNOTATION_TABLE_ID'])
    institution_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['INSTITUTION_TABLE_ID'])
    
    existing_records = image_table.all(
        formula=match({"name": image_name, "location": image_path.rstrip("/")})
    )

    if image_type in ["human_segmentation", "ml_segmentation"]:
        value_type = "label"
    else:
        value_type = "scalar"

    input_zarr = read(image_path.rstrip("/"))
    if isinstance(input_zarr, zarr.Group):
        zg = input_zarr
        z_arr_name = "s0"
    else:
        zg_path, z_arr_name = os.path.split(image_path.rstrip("/"))
        zg = read(zg_path)

    scale = zg.attrs["multiscales"][0]["datasets"][0]["coordinateTransformations"][0][
        "scale"
    ]
    offset = zg.attrs["multiscales"][0]["datasets"][0]["coordinateTransformations"][1][
        "translation"
    ]
    shape = zg[z_arr_name].shape

    try:
        fibsem_imaging = [fibsem_table.all(formula=match({"name": ds_name}))[0]["id"]]
    except:
        fibsem_imaging = []

    try:
        annotation = [
            annotation_table.all(formula=match({"name": image_name}))[0]["id"]
        ]
    except:
        annotation = []

    record_to_upsert = {
        "name": image_name,
        "collection": [collection_table.all(formula=match({"id": ds_name}))[0]["id"]],
        "location": image_path.rstrip("/"),
        "format": "zarr",
        "title" : image_title,
        "institution" : [institution_table.all(formula=match({"name": institution}))[0]["id"]],
        "image_type": image_type,
        "value_type": value_type,
        "size_x_pix": shape[2],
        "size_y_pix": shape[1],
        "size_z_pix": shape[0],
        "resolution_x_nm": scale[2],
        "resolution_y_nm": scale[1],
        "resolution_z_nm": scale[0],
        "offset_x_nm": offset[2],
        "offset_y_nm": offset[1],
        "offset_z_nm": offset[0],
        "fibsem_imaging": fibsem_imaging,
        "annotation": annotation,
    }

    if len(existing_records) > 2:
        raise ValueError("Multiple records with matching input image name found")

    if not existing_records:
        image_table.create(record_to_upsert)
    elif len(existing_records) == 1:
        image_table.update(existing_records[0]["id"], record_to_upsert)


In [ ]:
for p in zarr_paths:
    ds_name = p.split('/')[0].split('.')[0]
    image_name = 'nuc'
    image_path = os.path.join('/nrs/cellmap/data/',ds_name, p)
    image_title = 'Nucleus'

    upsert_image(at_api,
                    ds_name,
                    image_name,
                    image_path,
                    image_title,
                    "ml_segmentation")